## Adding Memory in Agentic Graph
### Agent with memory

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict

from langgraph.graph import StateGraph,START,END
from langgraph.graph.message import add_messages
from langchain_groq import ChatGroq

import os
from dotenv import load_dotenv
load_dotenv()

# loading the llm
api_key = os.getenv("GROQ_API")
llm = ChatGroq(model = "gemma2-9b-it", api_key = api_key)

# Loading the tavily tool
from langchain_tavily import TavilySearch
tavily_api_key = os.getenv("TAVILY_API")

tool = TavilySearch(max_results = 2)

In [ ]:
#Defining custom function
def multipy(a:int,b:int)->int: # the doc string is very important
    """Multiply a and b # example of document string

    Args:
        a (int) : first int
        b (int) : second int

    Returns:
        int: output int
    """
    return(a*b)

#Creating the tools list 
tools = [tool,multipy]

#binding llm with tools
llm_with_tools = llm.bind_tools(tools)